In [1]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize

Importing new merged DataFrame.


In [2]:
client = MongoClient('localhost', 27017)
collection = client['companies'].companies_v2_2
data = pd.DataFrame(list(collection.find()))
data = data[['name', 'founded_year', 'category_code', 'number_of_employees', 'rounds', 'total_raised_MnUSD', 'round_average_MnUSD', 
             'type: 1: startup, 2: small/mid corp, 3:big corp', 'latitude', 'longitude', 'geojson']]
data.head()

,name,founded_year,category_code,number_of_employees,rounds,total_raised_MnUSD,round_average_MnUSD,"type: 1: startup, 2: small/mid corp, 3:big corp",latitude,longitude,geojson
0,Geni,2006,web,18,3.0,16.5,5.5,2,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,Twitter,2006,social,1300,7.0,760.2,108.6,3,37.776805,-122.416924,"{'type': 'Point', 'coordinates': [-122.4169244..."
2,Facebook,2004,social,5299,10.0,2325.7,232.6,3,37.416050,-122.151801,"{'type': 'Point', 'coordinates': [-122.151801,..."
3,MeetMoi,2007,social,15,3.0,5.6,1.9,2,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ..."
4,Facebook,2004,social,5299,10.0,2325.7,232.6,3,40.755716,-73.979247,"{'type': 'Point', 'coordinates': [-73.9792469,..."


Defining a function geo query to get number of near companies of each company (3km around).


In [3]:
def get_near_companies(geojson):
    nearcomps = pd.DataFrame(list(collection.find(
        {'geojson': 
         { '$near': 
          {'$geometry': geojson, '$maxDistance': 3000}}}, 
        {'name'})))
    return len(nearcomps)


Function geo query to get the sum of the total number employees of the companies around each company of the DataFrame.


In [4]:
def get_near_employees(geojson):
    nearemps = pd.DataFrame(list(collection.find(
        {'geojson': 
         { '$near': 
          {'$geometry': geojson, '$maxDistance': 3000}}})))
    return sum(nearemps['number_of_employees'])


Function to get ratio startup/big corp of each company of DataFrame. This is a special ratio where equal number of startups for each big corp is valued at '2'.


In [5]:
def get_ratio(geojson):
    ratio = pd.DataFrame(list(collection.find(
        {'geojson': 
         { '$near': 
          {'$geometry': geojson, '$maxDistance': 3000}}})))
    return sum(ratio['type: 1: startup, 2: small/mid corp, 3:big corp'])/len(ratio)


Function to get ratio startup/big corp of each company of DataFrame. This is a special ratio where equal number of startups for each big corp is valued at '2'.

In [6]:
def get_investments(geojson, tipo=1):
    investments = pd.DataFrame(list(collection.find(
        {'geojson': 
         { '$near': 
          {'$geometry': geojson, '$maxDistance': 3000}}})))
    investments = investments[investments['type: 1: startup, 2: small/mid corp, 3:big corp']==1]    
    return investments.total_raised_MnUSD.sum()

Calling all functions.

In [7]:
data['near_companies'] = data['geojson'].apply(lambda x: get_near_companies(x))
data['employees'] = data['geojson'].apply(lambda x: get_near_employees(x))
data['ratio'] = data['geojson'].apply(lambda x: get_ratio(x))
data['investments'] = data['geojson'].apply(lambda x: get_investments(x))

Here we drop all the points with less than 50 companies around and we create the column ratio2 with the distance to the startup/big corp ratio desired. 


In [8]:
min_near_companies = 20
data_final = data[(data['near_companies'] > min_near_companies)]
data_final['ratio_distance'] = ((data_final['ratio']-2)**2)**(1/2)

/usr/local/Cellar/ipython/7.4.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Getting a new DataFrame with all values with a distance to perfect equallity of more than 0.2 points.

In [9]:
max_ratio_distance = 0.2
data_final = data_final[(data_final['ratio_distance'] < max_ratio_distance)]
data_final = data_final.sort_values(by='investments', ascending=False)
data_final.index=range(len(data_final))
data_final.head()


,name,founded_year,category_code,number_of_employees,rounds,total_raised_MnUSD,round_average_MnUSD,"type: 1: startup, 2: small/mid corp, 3:big corp",latitude,longitude,geojson,near_companies,employees,ratio,investments,ratio_distance
0,Twitter,2006,social,1300,7.0,760.2,108.6,3,37.776805,-122.416924,"{'type': 'Point', 'coordinates': [-122.4169244...",68,4969,1.941176,37.1,0.058824
1,Tagged,2004,social,162,4.0,23.7,5.9,2,37.775300,-122.418600,"{'type': 'Point', 'coordinates': [-122.4186, 3...",66,4949,1.939394,37.1,0.060606
2,GuardianEdge Technologies,2002,software,85,2.0,16.0,8.0,2,37.778613,-122.395289,"{'type': 'Point', 'coordinates': [-122.395289,...",66,4956,1.939394,37.1,0.060606
3,GitHub,2008,software,106,1.0,100.0,100.0,3,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,...",66,4949,1.939394,37.1,0.060606
4,Three Rings,2001,web,30,2.0,7.0,3.5,2,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,...",66,4949,1.939394,37.1,0.060606


Exporting the DataFrame to a CSV

In [10]:
def toCSV(data):
    data.to_csv('final.csv')

toCSV(data_final)

In [11]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 16 columns):
name                                               111 non-null object
founded_year                                       111 non-null int64
category_code                                      111 non-null object
number_of_employees                                111 non-null int64
rounds                                             100 non-null float64
total_raised_MnUSD                                 111 non-null float64
round_average_MnUSD                                111 non-null float64
type: 1: startup, 2: small/mid corp, 3:big corp    111 non-null int64
latitude                                           111 non-null float64
longitude                                          111 non-null float64
geojson                                            111 non-null object
near_companies                                     111 non-null int64
employees                                 